In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/fasttext-crawl-300d-2m/crawl-300d-2M.vec
/kaggle/input/news-summary/news_summary.csv
/kaggle/input/news-summary/news_summary_more.csv


In [2]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
import numpy as np
from nltk import word_tokenize
from numpy import array
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Embedding
from keras.utils.vis_utils import plot_model
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.preprocessing.sequence import pad_sequences
import re,codecs
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [3]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [4]:
import pandas as pd
df = pd.read_csv("../input/news-summary/news_summary.csv",encoding = "ISO-8859-1")
#news_summary_more = pd.read_csv("../input/news-summary/news_summary_more.csv",encoding = "ISO-8859-1")
columns=df.columns
new_df=df[['ctext','text']]
#print(df.describe())
#print(new_df.head())
#print("Description:",new_df['ctext'][0])
#print("Summary:",new_df['text'][0])
#raw=pd.read_csv("../input/datafornewssummarisation/summary.csv",encoding = "ISO-8859-1")
#raw.to_csv(r'../working/datafornewssummarisation\summary.csv')
new_df.to_csv('summary.csv',encoding = "ISO-8859-1")


In [5]:
df=pd.read_csv("../working/summary.csv",encoding = "ISO-8859-1")
columns=df.columns

#print(columns)


#An example showing the data and the summary
#print("Text:",X[1])
#print("\n")
#print("Summary:",y[1])
#Text cleaning
#Rows having duplicates will be omitted
df.drop_duplicates(keep='first',inplace=True)

#Rows not having summary or document is deleted
df.dropna(axis=0,inplace=True)

In [6]:
    stop_words = set(stopwords.words('english')) 
    def text_cleaner(text):
        newString = text.lower()
        newString = BeautifulSoup(newString, "lxml").text
        newString = re.sub(r'\([^)]*\)', '', newString)
        newString = re.sub('"','', newString)
        newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
        newString = re.sub(r"'s\b","",newString)
        newString = re.sub("[^a-zA-Z]", " ", newString) 
        tokens = [w for w in newString.split() if not w in stop_words]
        long_words=[]
        for i in tokens:
            if len(i)>1:                  #removing short word
                long_words.append(i)   
        return (" ".join(long_words)).strip()

    cleaned_text = []
    for t in df['ctext']:
        cleaned_text.append(text_cleaner(t))
    #print('done')

    def summary_cleaner(text):
        newString = re.sub('"','', text)
        newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
        newString = re.sub(r"'s\b","",newString)
        newString = re.sub("[^a-zA-Z]", " ", newString)
        newString = newString.lower()
        tokens=newString.split()
        newString=''
        for i in tokens:
            if len(i)>=3:                                 
                newString=newString+i+' '  
        return newString

    #Call the above function
    cleaned_summary = []
    for t in df['text']:
        cleaned_summary.append(summary_cleaner(t))
    

    df['cleaned_text']=cleaned_text
    df['cleaned_summary']=cleaned_summary
    df.replace('', np.nan, inplace=True)
    df.dropna(axis=0,inplace=True)
    print("done")

done


In [7]:
#dataset_cleaning()
df.head()

,Unnamed: 0,ctext,text,cleaned_text,cleaned_summary
0,0,The Daman and Diu administration on Wednesday ...,The Administration of Union Territory Daman an...,daman diu administration wednesday withdrew ci...,the administration union territory daman and d...
1,1,"From her special numbers to TV?appearances, Bo...",Malaika Arora slammed an Instagram user who tr...,special numbers tv appearances bollywood actor...,malaika arora slammed instagram user who troll...
2,2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,indira gandhi institute medical sciences patna...,the indira gandhi institute medical sciences i...
3,3,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,Lashkar-e-Taiba's Kashmir commander Abu Dujana...,lashkar taiba kashmir commander abu dujana kil...,lashkar taiba kashmir commander abu dujana who...
4,4,Hotels in Mumbai and other Indian cities are t...,Hotels in Maharashtra will train their staff t...,hotels mumbai indian cities train staff spot s...,hotels maharashtra will train their staff spot...


In [8]:
cleaned_text =np.array(df['cleaned_text'])
cleaned_summary=np.array(df['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=63 and len(cleaned_text[i].split())<=600):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
data=pd.DataFrame({'text':short_text,'summary':short_summary})

In [9]:
data.head()

,text,summary
0,daman diu administration wednesday withdrew ci...,the administration union territory daman and d...
1,special numbers tv appearances bollywood actor...,malaika arora slammed instagram user who troll...
2,indira gandhi institute medical sciences patna...,the indira gandhi institute medical sciences i...
3,lashkar taiba kashmir commander abu dujana kil...,lashkar taiba kashmir commander abu dujana who...
4,hotels mumbai indian cities train staff spot s...,hotels maharashtra will train their staff spot...


In [10]:
data['summary']=data['summary'].apply(lambda x:'sostok '+x+' eostok')
data.head()

,text,summary
0,daman diu administration wednesday withdrew ci...,sostok the administration union territory dama...
1,special numbers tv appearances bollywood actor...,sostok malaika arora slammed instagram user wh...
2,indira gandhi institute medical sciences patna...,sostok the indira gandhi institute medical sci...
3,lashkar taiba kashmir commander abu dujana kil...,sostok lashkar taiba kashmir commander abu duj...
4,hotels mumbai indian cities train staff spot s...,sostok hotels maharashtra will train their sta...


In [11]:
X_train,X_test,y_train,y_test=train_test_split(np.array(data['text']),
    np.array(data['summary']),test_size=0.1,random_state=0,shuffle=True)

In [12]:
    #This function  will be used to preprocess the data to a form that can be fed into the neural network
    #Subparts:
    ##1.Find the most_probable_length of the sentences of text_type and summary type 
    ##(Hint: you may plot the distribution of lengths of sentences)
    """    
    text_count = []
    summary_count = []

    for t in df_processed['cleaned_text']:
        text_count.append(len(t.split()))
    for t in df_processed['cleaned_summary']:
        summary_count.append(len(t.split()))
     """
      
   #graph_df= pd.DataFrame()
   #graph_df['text']=text_count
    #df2=pd.DataFrame()
  # graph_df['summary']=summary_count
    #df2['summary']=summary_count
  # graph_df.hist(bins = 5,range = [0,600])
  # df2.hist(bins = 5,range = [50,63])
   #plt.show()
    max_sum_len=63
    max_text_len=600
    

    ##2.Tokenizing the data
    X_tokenizer = Tokenizer()
    X_tokenizer.fit_on_texts(list(X_train))

    thresh=4
    cnt=0
    tot_cnt=0
    freq=0
    tot_freq=0

    for key,value in X_tokenizer.word_counts.items():
        tot_cnt=tot_cnt+1
        tot_freq=tot_freq+value
        if(value<thresh):
            cnt=cnt+1
            freq=freq+value
    
    X_tokenizer=Tokenizer(num_words=tot_cnt)
    X_tokenizer.fit_on_texts(list(X_train))
    X_train_seq = X_tokenizer.texts_to_sequences(X_train)
    X_test_seq=X_tokenizer.texts_to_sequences(X_test)
    X_train=pad_sequences(X_train_seq,maxlen=max_text_len,padding='post')
    X_test= pad_sequences(X_test_seq,maxlen=max_text_len,padding='post')   
    X_voc=X_tokenizer.num_words +1
    
    y_tokenizer = Tokenizer()
    y_tokenizer.fit_on_texts(list(y_train))
    thresh=6
    cnt=0
    tot_cnt=0
    freq=0
    tot_freq=0

    for key,value in y_tokenizer.word_counts.items():
        tot_cnt=tot_cnt+1
        tot_freq=tot_freq+value
        if(value<thresh):
            cnt=cnt+1
            freq=freq+value
    y_tokenizer=Tokenizer(num_words=(tot_cnt))
    y_tokenizer.fit_on_texts(list(y_train))
    y_train_seq = y_tokenizer.texts_to_sequences(y_train)
    y_test_seq=y_tokenizer.texts_to_sequences(y_test)
    y_train=pad_sequences(y_train_seq,maxlen=max_sum_len,padding='post')
    y_test= pad_sequences(y_test_seq,maxlen=max_sum_len,padding='post')  
    y_voc=y_tokenizer.num_words +1
    
    print(y_tokenizer.word_counts['sostok'],len(y_train))
    
    ind=[]
    for i in range(len(y_train)):
        cnt=0
        for j in y_train[i]:
            if j!=0:
                cnt=cnt+1
        if(cnt==2):
            ind.append(i)

    y_train=np.delete(y_train,ind, axis=0)
    X_train=np.delete(X_train,ind, axis=0)
    
    ind=[]
    for i in range(len(y_test)):
        cnt=0
        for j in y_test[i]:
            if j!=0:
                cnt=cnt+1
        if(cnt==2):
            ind.append(i)

    y_test=np.delete(y_test,ind, axis=0)
    X_test=np.delete(X_test,ind, axis=0)

3880 3880


In [13]:
def preprocessing():
##4.Introducing embeddings( We'll be using the Fasttext embeddings)
    embeddings_index = {}
    f = open('../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

    word_index = X_tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    print("done")
    
    return embedding_matrix

In [14]:
#X_voc,y_voc,x_tr,x_val,y_tr,y_val,X_tokenizer,y_tokenizer=preprocessing(X_train,X_test,y_train,y_test,df_processed)
embedding_matrix=preprocessing()
#y_tokenizer.word_counts['sostok'],len(y_tr)

done


In [15]:
max_sum_len=63
max_text_len=600
X_voc,embedding_matrix.shape

(42827, (42827, 300))

In [16]:
def model(embedding_matrix):
    from keras import backend as k
    k.clear_session()
    
    encoder_inputs=Input(shape=(max_text_len,))
    encoder_emb=Embedding(X_voc,300,weights=[embedding_matrix],trainable=True)(encoder_inputs)
    encoder_lstm1 = LSTM(300,return_sequences=True,return_state=True)
    encoder_output1, state_h1, state_c1 = encoder_lstm1(encoder_emb)
    encoder_lstm2 = LSTM(300,return_sequences=True,return_state=True)
    encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)
    encoder_lstm3=LSTM(300, return_state=True, return_sequences=True)
    encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)


    decoder_inputs = Input(shape=(None,))
    dec_emb_layer = Embedding(y_voc, 300,trainable=True)
    dec_emb = dec_emb_layer(decoder_inputs)
    decoder_lstm = LSTM(300, return_sequences=True, return_state=True)
    decoder_outputs,decoder_fwd_state, decoder_back_state =decoder_lstm(dec_emb,initial_state=[state_h, state_c])
    decoder_dense =TimeDistributed(Dense(y_voc, activation='softmax'))
    decoder_outputs =decoder_dense(decoder_outputs)

# Define the model 
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    model.summary()
    return model,encoder_inputs,encoder_outputs,state_h,state_c,decoder_inputs,dec_emb_layer,decoder_lstm,decoder_dense

In [17]:
m,encoder_inputs,encoder_outputs,state_h,state_c,decoder_inputs,dec_emb_layer,decoder_lstm,decoder_dense=model(embedding_matrix)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 600)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 600, 300)     12848100    input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 600, 300), ( 721200      embedding_1[0][0]                
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
____________________________________________________________________________________________

Model is built and summary shown above

In [18]:
def model_accuracy(model):
    #Function to check how the model works on the test data
    model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
    from keras.callbacks import EarlyStopping
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
    history=model.fit([X_train,y_train[:,:-1]], 
                      y_train.reshape(y_train.shape[0],y_train.shape[1], 1)
                      [:,1:] ,epochs=50,callbacks=[es],
                      batch_size=128, validation_data=([X_test,y_test[:,:-1]],
                                                       y_test.reshape(y_test.shape[0],y_test.shape[1], 1)[:,1:]))

In [19]:
model_accuracy(m)

/opt/conda/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3880 samples, validate on 432 samples
Epoch 1/50
3880/3880 [==============================] - 71s 18ms/step - loss: 7.0167 - val_loss: 5.9437
Epoch 2/50
3880/3880 [==============================] - 69s 18ms/step - loss: 6.1008 - val_loss: 5.5185
Epoch 3/50
3880/3880 [==============================] - 69s 18ms/step - loss: 5.8917 - val_loss: 5.4534
Epoch 4/50
3880/3880 [==============================] - 67s 17ms/step - loss: 5.8039 - val_loss: 5.4368
Epoch 5/50
3880/3880 [==============================] - 67s 17ms/step - loss: 5.7363 - val_loss: 5.3738
Epoch 6/50
3880/3880 [==============================] - 68s 18ms/step - loss: 5.6650 - val_loss: 5.3252
Epoch 7/50
3880/3880 [==============================] - 68s 17ms/step - loss: 5.5938 - val_loss: 5.2948
Epoch 8/50
3880/3880 [==============================] - 67s 17ms/step - loss: 5.5114 - val_loss: 5.2435
Epoch 9/50
3880/3880 [==============================] - 68s 18ms/step - loss: 5.4257 - val_loss: 5.1913
Epoch 10/50
3880/

 Model accuracy

In [20]:
def predict():
    reverse_target_word_index=y_tokenizer.index_word
    reverse_source_word_index=X_tokenizer.index_word
    target_word_index=y_tokenizer.word_index
    latent_dim=300
    # Encode the input sequence to get the feature vector
    encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
    dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
    decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
    decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
    decoder_model = Model([decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, 
                                              decoder_state_input_c],[decoder_outputs2] + [state_h2, state_c2])
    
    def decode_sequence(input_seq):
    # Encode the input as state vectors.
        e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
        target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
        target_seq[0, 0] = target_word_index['sostok']

        stop_condition = False
        decoded_sentence = ''
        while not stop_condition:
      
            output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_token = reverse_target_word_index[sampled_token_index]
        
            if(sampled_token!='eostok'):
                decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
            if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_sum_len-1)):
                stop_condition = True

        # Update the target sequence (of length 1).
            target_seq = np.zeros((1,1))
            target_seq[0, 0] = sampled_token_index

        # Update internal states
            e_h, e_c = h, c

        return decoded_sentence
    
    def seq2summary(input_seq):
        newString=''
        for i in input_seq:
            if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
                newString=newString+reverse_target_word_index[i]+' '
        return newString

    def seq2text(input_seq):
        newString=''
        for i in input_seq:
            if(i!=0):
                newString=newString+reverse_source_word_index[i]+' '
        return newString
    
    for i in range(0,4):
        print("Review:",seq2text(X_train[i]))
        print("Original summary:",seq2summary(y_train[i]))
        print("Predicted summary:",decode_sequence(X_train[i].reshape(1,max_text_len)))
        print("\n")

In [21]:
predict()

Review: new delhi may indian railways install cctv surveillance system stations nirbhaya fund approved cost rs crore strengthen security rail premises tenders floated shortly install high definition cctv cameras stations across country ensure round clock security passengers including women centre created nirbhaya fund rs crore corpus union budget support initiatives government ngos working towards protecting dignity ensuring safety women country project envisages installation cameras platforms waiting areas control rooms stations monitor cctv footages trained rpf personnel constantly station master also given access monitor cctv camera footage part project undertaken public transporter said senior railway ministry official railways stations currently stations already provided cctv cameras helping us great way signboard reading watch preventive effect criminal minds official said besides keeping constant watch cctv surveillance system helps post incident crime investigation railways aim